In [3]:
import polars as pl
from polars import col
pl.__version__

'0.13.43'

### Summary

The major lessons here are:

1. Getting comfortable with polars
1. The power in using `extract` combined with `when/then/otherwise` for data cleaning
1. How to write code that's easy to read for cleaning data that's hard to read

### Cleaning the hospitals dataset -- motivation

The hospitals dataset at DoltHub contains 300M rows of prices. Each price is given a couple of codes (a primary `code` and an `internal_revenue_code`) which refer to how the procedure is billed. Billing codes are the standard for comparing hospital prices. If we know that a hospital charges 2,000 for code `59410`, you can be pretty sure that you (or your insurer) are going to be paying about that price for a normal childbirth.

#### Where the codes are in our dataset

Hospitals, according to the CMS law, are required to post a chargemaster with a hospital-generic code. As they should be: if hospitals only used proprietary coding for their procedures there would be no way to compare prices between hospitals. For an apples-to-apples comparison, we all need to use apples. Typically these apples are the CPT codes ("current procedural terminology") which follow a specific pattern. 

We'll do our best to extract CPT codes (as well as DRG and NDC codes) from the chargemasters as much as we reasonably can. Remember that the CMS law requires that these chargemasters be machine readable. If we can't figure out what the code is supposed to be from the chargemaster, then it's not machine readable. It's junk.


#### How codes were entered

Participants typically put the most generic code in the `code` column. This would be where the CPT code went. If there was a second code, it went in the `internal_revenue_code` column. A third code would have gone in the `code_disambiguator` column, but this turns out to not have been necessary.

However, because the codes are mixed -- some contain pharmacy codes, random codes, etc. -- it is a bit difficult to work with the data. Plus, it's not clear how many of our rows even have valid codes.

So I wrote a short cleaning pipeline to figure out the coding for each row, and see what fraction of rows were coded in some machine readable way.

### Making the pipeline: custom pipes

`pipes` are an easy way to chain data transformations together. In polars/pandas, it looks something like:

```python
out_df = in_df.pipe(transformation, **params).pipe(another_transformation, **params)
```

The first transformation we'll need is one that extracts the CPT codes from the codes column.

In [935]:
def extract_cpt(df: pl.DataFrame) -> pl.DataFrame:
    '''
    Extract CPT singlet codes
    '''
    
    # do some transformations
    
    return df

### A test dataframe

We'll use to test out our transformations.

In [1381]:
testdf = pl.DataFrame({'code': ['  CPT/HCPC 0206009123 ', 'NDC 0011-1231-12-', 'CPT 99899 00123', 'MS-DRG 1101112', 
                                'CPT 123', '12341-12345', '  CPT12345', 'NDC 0012312311',
                                'MS123', '319',],
                       'description': ['some procedure', 'a drug 10mg', 'heart surgery', 'a surgery',
                                       'a short cpt code', 'a range of cpt codes', 'NDC 123456-1281', 'another drug', 
                                       'proc', 'heart surgery']})
print(testdf)

shape: (10, 2)
┌────────────────────────┬─────────────────┐
│ code                   ┆ description     │
│ ---                    ┆ ---             │
│ str                    ┆ str             │
╞════════════════════════╪═════════════════╡
│   CPT/HCPC 0206009123  ┆ some procedure  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ NDC 0011-1231-12-      ┆ a drug 10mg     │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ CPT 99899 00123        ┆ heart surgery   │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ MS-DRG 1101112         ┆ a surgery       │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ ...                    ┆ ...             │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│   CPT12345             ┆ NDC 123456-1281 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ NDC 0012312311         ┆ another drug    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ MS123                  ┆ proc            │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 319                    ┆ heart surgery

### Simple cleaning

At the very least, we can write a quick function to clean the code column. This'll make it easier to do regex matching and joining later.

In [1383]:
def clean(df: pl.DataFrame, colname) -> pl.DataFrame:
    return df.with_column(
        col(colname).str.strip().str.replace_all('^-|-$', '')
    )
    
print(testdf.pipe(clean, 'code'))

shape: (10, 2)
┌─────────────────────┬─────────────────┐
│ code                ┆ description     │
│ ---                 ┆ ---             │
│ str                 ┆ str             │
╞═════════════════════╪═════════════════╡
│ CPT/HCPC 0206009123 ┆ some procedure  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ NDC 0011-1231-12    ┆ a drug 10mg     │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ CPT 99899 00123     ┆ heart surgery   │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ MS-DRG 1101112      ┆ a surgery       │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ ...                 ┆ ...             │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ CPT12345            ┆ NDC 123456-1281 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ NDC 0012312311      ┆ another drug    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ MS123               ┆ proc            │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 319                 ┆ heart surgery   │
└─────────────────────┴─────────────────┘


### The power of regex

We'll hunt for the CPT codes in the codes column. We'll use a few heuristics for finding them. To get those heuristics, we'll need to know what kind of junk is present in the database already, and how to distinguish that junk from valid codes to the greatest possible extent.

#### Heuristic 1: There's only one code type in each cell

If the cell contains the string 'NDC', 'MS-DRG', or 'CDM', it is overwhelmingly likely to *not* contain a CPT code.

#### Heuristic 2: A specific set of 5-char codes

CPT codes come mainly in two flavors:

1. CPT I: 5 digit numerical code from 00100 to 99499
2. CPT II: 5 digit alphanumerical code like `any('ACEGJKLMQTV')` + 4 digits or 4 digits + `any('ACEGJKLMQTV')`

We will hunt for explicitly these patterns.

In [1256]:
def extract_cpt(df: pl.DataFrame) -> pl.DataFrame:
    '''
    Extract CPT singlet codes
    '''
    
    cpt_pats = ['\d{5}',       # simplified CPT I
                '[A-Z]\d{4}',  # simplified CPT II
                '\d{4}[A-Z]']  # simplified CPT II
    
    # to "extract" a pattern, it needs to be in parenthesis
    # and to extract multiple patterns with OR, they need to
    # have a bar char between them. A valid extraction string
    # looks something like 'abc(pat)'
    
    extraction_str = '|'.join([f'({pat})' for pat in cpt_pats])
    
    return df.with_column(
        
        pl.when(
            
            # Heuristic 1
            ~col('code').str.contains('NDC|MS|CDM')
            
        ).then(
            
            # Heuristic 2
            col('code').str.extract(extraction_str)
            
        ).otherwise(None).alias('extracted_cpt')
    )

testdf.pipe(extract_cpt)

code,description,extracted_cpt
str,str,str
"""CPT/HCPC 02060...","""some procedure...","""02060"""
"""NDC 0011-1231-...","""a drug 10mg""",null
"""CPT 99899 0012...","""heart surgery""","""99899"""
"""MS-DRG 1101112...","""a surgery""",null
"""CPT 123""","""a short cpt co...",null
"""12341-12345""","""a range of cpt...","""12341"""
"""CPT12345""","""NDC 123456-128...",null
"""NDC 0012312311...","""another drug""",null
"""MS123""","""proc""",null


We right away see some flaws here. 

1. Because there can be multiple CPT codes in a cell, we'll need a way to extract them all
2. Sometimes we're extracting codes that we shouldn't be (row 3 is not a valid CPT code)
3. The last row, which is a range of CPT codes, should probably be handled separately. Plus, these "ranges" might actually be longer codes with dashes in the middle (unless they're specifically marked as CPT). Let's filter them out for now.
4. 99899 isn't really a valid CPT code. More on that later.

### Solution addressing (1): `str.extract_all` to the rescue

This undocumented polars function allows us to extract all the CPT codes in a cell in one stroke.

### Solution addressing (2): regex word boundaries

By putting `\b` on either side of a word, we can force our five-char codes to not be part of a larger digit string.

### Solution addressing (3): eliminate dashed digit/char strings with a `str.replace_all`

We'll simply eliminate those patterns entirely before we do regex matching. (We could do this with negative lookaheads, but these aren't implemented in the rust regex parser.)

In [1257]:
def extract_cpt(df: pl.DataFrame) -> pl.DataFrame:
    '''
    Extract CPT singlet codes
    '''
    
    cpt_pats = ['\d{5}',       # simplified CPT I
                '[A-Z]\d{4}',  # simplified CPT II
                '\d{4}[A-Z]']  # simplified CPT II
    
    # to "extract" a pattern, it needs to be in parenthesis
    # and to extract multiple patterns with OR, they need to
    # have a bar char between them. A valid extraction string
    # looks something like 'abc(pat)'
    
    extraction_str = '|'.join([fr'\b({pat})\b' for pat in cpt_pats])
    
    return df.with_column(
        
        col('code').str.replace_all(r'\b\d{5}-\d{5}\b', '').alias('code_cleaned')
        
    ).with_column(
        
        pl.when(
            
            # Heuristic 1
            ~col('code_cleaned').str.contains('NDC|MS|CDM')
            
        ).then(
            
            # Heuristic 2
            col('code_cleaned').str.extract_all(extraction_str)
            
        ).otherwise(None).alias('extracted_cpt')
    )

testdf.pipe(extract_cpt)

code,description,code_cleaned,extracted_cpt
str,str,str,list[str]
"""CPT/HCPC 02060...","""some procedure...","""CPT/HCPC 02060...",null
"""NDC 0011-1231-...","""a drug 10mg""","""NDC 0011-1231-...",null
"""CPT 99899 0012...","""heart surgery""","""CPT 99899 0012...","[""99899"", ""00123""]"
"""MS-DRG 1101112...","""a surgery""","""MS-DRG 1101112...",null
"""CPT 123""","""a short cpt co...","""CPT 123""",null
"""12341-12345""","""a range of cpt...","""""",null
"""CPT12345""","""NDC 123456-128...","""CPT12345""",null
"""NDC 0012312311...","""another drug""","""NDC 0012312311...",null
"""MS123""","""proc""","""MS123""",null


Cool! We're on our way to doing a good cleanup on the dataset.

### the "raw string" r''

If you've never seen `r'some string'` before, the `r` just stops what's inside the quotations from being escaped with slashes. So, for example, `r'\n'` is *really* slash n, as opposed to `'\n'`, which is a newline.

In [1258]:
print(r'This is \n not a newline')
print('This is a \n newline')

This is \n not a newline
This is a 
 newline


This prevents the `\b` from being parsed as something else, so that the regex engine can work with it properly (and treat it as a word boundary.) Without the `r`, you need double slashes.

In [1259]:
print(r'A raw string: \b and \\b')
print('A regular string \\b and \b')

A raw string: \b and \\b
A regular string \b and


### Convenience functions

#### `in_col`
Inside `str.contains` should be a list of ORs. 

```python
# Heuristic 1
~col('code_cleaned').str.contains('NDC|MS|CDM')
```

But this will be cumbersome to write once we have a lot of stuff to put in there. It would be nice if we could just pass a list of strings directly to a function.

In [1260]:
def in_col(colname, words): 
    return col(colname).str.contains('|'.join(words.split()))

def extract_all_from(colname, words):
    words_str = '|'.join([fr'\b{word}\b' for word in words.split()])
    return col(colname).str.extract_all(words_str)

def clear_all_from(colname, words):
    words_str = '|'.join([fr'{word}' for word in words.split()])
    return col(colname).str.replace_all(words_str, '')

This allows us to replace, for example:
```python
# ~col('code_cleaned').str.contains('NDC|MS|CDM') 
in_col('code_cleaned', 'NDC MS CDM')

# cpt_pats = ['\d{5}', '[A-Z]\d{4}', '\d{4}[A-Z]'] 
# col('code_cleaned').str.extract_all(extraction_str)
cpt_pats = '\d{5} [A-Z]\d{4} \d{4}[A-Z]'
extract_all_from('code_cleaned', cpt_pats)
```

which looks neater to me and will get easier to work with.

### Writing the extraction function with our new functions
results in a much cleaner transformation.

In [1261]:
def extract_cpt(df: pl.DataFrame) -> pl.DataFrame:
    '''
    Extract CPT singlet codes
    '''
    
    cpt_pats = '\d{5} [A-Z]\d{4} \d{4}[A-Z]'
    
    return df.with_column(
        
        clear_all_from('code', '\d{5}-\d{5}').alias(f'code_cleaned')
        
    ).with_column(
        
        pl.when(
            
            # Heuristic 1
            ~in_col('code_cleaned', 'NDC MS CDM')
            
        ).then(
            
            # Heuristic 2
            extract_all_from('code_cleaned', cpt_pats)
            
        ).otherwise(None).alias('extracted_cpt')
    )

testdf.pipe(extract_cpt)

code,description,code_cleaned,extracted_cpt
str,str,str,list[str]
"""CPT/HCPC 02060...","""some procedure...","""CPT/HCPC 02060...",null
"""NDC 0011-1231-...","""a drug 10mg""","""NDC 0011-1231-...",null
"""CPT 99899 0012...","""heart surgery""","""CPT 99899 0012...","[""99899"", ""00123""]"
"""MS-DRG 1101112...","""a surgery""","""MS-DRG 1101112...",null
"""CPT 123""","""a short cpt co...","""CPT 123""",null
"""12341-12345""","""a range of cpt...","""""",null
"""CPT12345""","""NDC 123456-128...","""CPT12345""",null
"""NDC 0012312311...","""another drug""","""NDC 0012312311...",null
"""MS123""","""proc""","""MS123""",null


### Continuing on
Let's add the detail needed to make this extraction more realistic.

1. We'll only use valid 5 digit CPT codes (so we'll need a more advanced regex). I used https://3widgets.comcodes to get a regex that wil capture numbers from 00100-99499, and added the alphanumerical ones manually.
2. It sometimes happens that codes are marked as CPT codes but don't fit the regex pattern. Any time there's a CPT/HCPCS string in the cell we're going to keep all the (reasonable) codes that are in it, even if they don't match the exact pattern of being a 5 digit string. They might have just had some zeros lopped off the front or something like that.

In [1262]:
def extract_cpt(df: pl.DataFrame) -> pl.DataFrame:
    '''
    Extract CPT singlet codes
    '''
    
    cpt_pats = '''0010[0-9] 001[1-9][0-9] 00[2-9][0-9]{2} 
                0[1-9][0-9]{3} [1-8][0-9]{4} 9[0-8][0-9]{3} 
                99[0-4][0-9]{2} [ACEGJKLMQTV]\d{4} \d{4}[ACEGJKLMQTV]'''
    
    return df.with_column(
        
        clear_all_from('code', '\\b\w{5}-\w{5}\\b \\b?CPT\\b? \\b?HCPCS?\\b?').alias(f'code_cleaned'),
        
    ).with_column(
        
        pl.when(            
            # If we have a code to exclude...
            in_col('code', 'NDC MS CDM DRG PH\d+ ITM')
        ).then(
            # ...ignore the cell
            None
        ).when(
            # If we have an explicit CPT code...
            in_col('code', 'CPT HCPCS? CPT\s?/?HCPCS? HCPCS\s?/?CPT')
        ).then(
            # greedily take the candidates
            extract_all_from('code_cleaned', cpt_pats + ' ([A-Z]|\d){1,2}?\d{1,4}([A-Z]|\d){2} \\b\d{3}\\b')
        ).otherwise(
            # If we have no information about the code,
            # just select the good candidates
            extract_all_from('code_cleaned', cpt_pats)
            
        ).alias('extracted_cpt')
    ).drop('code_cleaned')

testdf.pipe(extract_cpt)

code,description,extracted_cpt
str,str,list[str]
"""CPT/HCPC 02060...","""some procedure...",null
"""NDC 0011-1231-...","""a drug 10mg""",null
"""CPT 99899 0012...","""heart surgery""","[""99899"", ""00123""]"
"""MS-DRG 1101112...","""a surgery""",null
"""CPT 123""","""a short cpt co...","[""123""]"
"""12341-12345""","""a range of cpt...",null
"""CPT12345""","""NDC 123456-128...","[""12345""]"
"""NDC 0012312311...","""another drug""",null
"""MS123""","""proc""",null


### Repeating for the other code types
We'll do the same thing to extract NDC codes and DRG codes cleanly from our dataset.

In [1263]:
def extract_ndc(df: pl.DataFrame) -> pl.DataFrame:
    '''
    Extract NDC singlet codes
    '''
    
    ndc_pats = '''\d{4}-\d{4}-\d{2} \d{5}-\d{3}-\d{2} \d{5}-\d{4}-\d{1}'''
    
    return df.with_column(
        clear_all_from('code', '_\d+').alias('code_cleaned')
    ).with_column(
        pl.when(            
            in_col('code', 'CPT HCPCS? CPT\s?/?HCPCS? HCPCS\s?/?CPT MS CDM DRG PH\d+ ITM')
        ).then(
            # if the code column doesn't have an NDC code, let's check the description column
            pl.when(
                in_col('description', 'NDC')
            ).then(
                extract_all_from('description', ndc_pats + ' ([0-9]|-){12} ([0-9]|-){11} \d{10}')
            ).otherwise(None)
        ).when(
            in_col('code', 'NDC')
        ).then(
            extract_all_from('code_cleaned', ndc_pats + ' ([0-9]|-){12} ([0-9]|-){11} \d{10}')
        ).otherwise(
            extract_all_from('code_cleaned', ndc_pats)
        ).alias('extracted_ndc')
    ).drop('code_cleaned')

testdf.pipe(extract_ndc)

code,description,extracted_ndc
str,str,list[str]
"""CPT/HCPC 02060...","""some procedure...",null
"""NDC 0011-1231-...","""a drug 10mg""","[""0011-1231-12""]"
"""CPT 99899 0012...","""heart surgery""",null
"""MS-DRG 1101112...","""a surgery""",null
"""CPT 123""","""a short cpt co...",null
"""12341-12345""","""a range of cpt...",null
"""CPT12345""","""NDC 123456-128...","[""123456-1281""]"
"""NDC 0012312311...","""another drug""","[""0012312311""]"
"""MS123""","""proc""",null


In [1246]:
def extract_drg(df: pl.DataFrame) -> pl.DataFrame:
    '''
    Extract DRG singlet codes
    '''
    
    drg_pats = '''\d{1,3}'''
    
    return df.with_column(
        clear_all_from('code', ' \\b?MS\\b? \\b?MS-DRG\\b? \\b?DRG\\b?').alias(f'code_cleaned'),
    ).with_column(
        pl.when(            
            in_col('code', 'CPT HCPCS? CPT\s?/?HCPCS? HCPCS\s?/?CPT NDC CDM PH\d+ ITM')
        ).then(None).when(
            in_col('code', drg_pats + ' MS-DRG DRG MS')
        ).then(
            extract_all_from('code_cleaned', drg_pats)
        ).otherwise(None).alias('extracted_drg')
    ).drop('code_cleaned')

testdf.pipe(extract_drg)

code,extracted_drg
str,list[str]
"""CPT/HCPC 02060...",null
"""NDC 0011-1231-...",null
"""CPT 99899 0012...",null
"""MS-DRG 1101112...",null
"""CPT 123""",null
"""12341-12345""",null
"""CPT12345""",null
"""NDC 0012312311...",null
"""MS123""","[""123""]"


## More problems

Some of the codes in the codes column are superficially CPT codes, but may also be ICD-10 codes. There's some overlap. We won't be able to distinguish them based on the code alone. The best we can do is provide multiple matches.

These ICD-10 codes can be mapped to DRG codes (diagnostic-related-groups). Since we already have a column for them, we'll convert those codes to DRG codes using a table I found on the CMS website.

https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AcuteInpatientPPS/MS-DRG-Classifications-and-Software.html

(I had to manually remove the header from appendix_B.txt to be able to read it.)

### The table which maps from ICD-10 to DRG

This table is kind of a mess. There are missing rows and there are code ranges, and importantly, the file is fixed-width formatted.

Thankfully, one of my friends (@abmyii on our Discord) pointed out that pandas has a `.read_fwf` method for reading fixed-width files.

In [1360]:
!head -n 20 appendix_B.txt

 I10 Dx  MDC DRG(s)      ICD-10-CM Description
 A000    06  371-373     Cholera due to Vibrio cholerae 01, biovar cholerae
 A001    06  371-373     Cholera due to Vibrio cholerae 01, biovar eltor
 A009    06  371-373     Cholera, unspecified
 A0100   18  867-869     Typhoid fever, unspecified
 A0101   18  867-869     Typhoid meningitis
 A0102   18  867-869     Typhoid fever with heart involvement
 A0103   18  867-869     Typhoid pneumonia
 A0104   18  867-869     Typhoid arthritis
 A0105   18  867-869     Typhoid osteomyelitis
 A0109   18  867-869     Typhoid fever with other complications
 A011    18  867-869     Paratyphoid fever A
 A012    18  867-869     Paratyphoid fever B
 A013    18  867-869     Paratyphoid fever C
 A014    18  867-869     Paratyphoid fever, unspecified
 A020    06  371-373     Salmonella enteritis
 A021    18  870-872     Salmonella sepsis
         25  974-976     
 A0220   18  867-869     Localized salmonella infection, unspecified
         25  974-976     


### How we'll deal with this

1. We'll use `.fill_null('forward')` to turn

```sql
 A021    18  870-872     Salmonella sepsis
         25  974-976     
```
into
```sql
 A021    18  870-872     Salmonella sepsis
 A021    25  974-976     Salmonella sepsis
```

2. We'll split the array on the hyphen, cast the strings to ints, and then create a range (with `pl.arange`) from the first to the last element

```sql
"371-373" --> [371, 372, 373] 
"951"     --> [951]
```

3. We'll explode on that new column, and convert numbers like 1 back to '001' with the `.zfill` method.

In [1414]:
import pandas as pd
s = (pl
     .DataFrame(pd.read_fwf('appendix_B.txt'), columns = ['dx', 'mdc', 'drgs', 'icd-10-description'])
     .fill_null('forward')
     .with_column(
         col('drgs').str.split('-').arr.eval(pl.element().cast(int, strict = False)).alias('drg_arr')
     ).with_column(
         pl.arange(col('drg_arr').arr.first(), col('drg_arr').arr.last() + 1)
     ).explode('drg_arr')
     .drop('drgs')
     .rename({'drg_arr':'drg'})
     .with_column(
         col('drg').cast(str).str.zfill(3)
     )
    )

s.sample(5)

dx,mdc,icd-10-description,drg
str,str,str,str
"""B480""","""18""","""Lobomycosis""","""867"""
"""M84479""","""08""","""Pathological f...","""559"""
"""S60947""","""09""","""Unspecified su...","""605"""
"""S52691""","""08""","""Other fracture...","""564"""
"""S50379""","""24""","""Other superfic...","""955"""


In [1384]:
lf = pl.scan_csv('../prices.csv', n_rows = 10_000_000, infer_schema_length = 0, encoding = 'utf8-lossy')

In [1385]:
lf.fetch(2)

cms_certification_num,payer,code,internal_revenue_code,units,description,inpatient_outpatient,price,code_disambiguator
str,str,str,str,str,str,str,str,str
"""010001""","""BLUE ADVANTAGE...","""HCPCS 82441""","""3018244101""",null,"""HC TEST FOR CH...","""UNSPECIFIED""","""279.02""","""NONE"""
"""010001""","""BLUE CROSS OF ...","""HCPCS 82441""","""3018244101""",null,"""HC TEST FOR CH...","""UNSPECIFIED""","""279.02""","""NONE"""


In [1386]:
df = (lf
      .pipe(clean, 'code')
      .pipe(extract_ndc)
      .pipe(extract_cpt)
      .pipe(extract_drg)
      .drop(['units', 'code_disambiguator'])
      .with_columns(
          pl.col(['cms_certification_num', 'inpatient_outpatient']).cast(pl.Categorical)
      )
     ).collect()

unmatched = df.filter(
    col('extracted_ndc').is_null()  & 
    col('extracted_drg').is_null() &
    col('extracted_cpt').is_null() 
)

In [1387]:
print(df.sample(5))
print(unmatched.sample(5))

shape: (5, 10)
┌────────────┬────────┬───────┬────────────┬─────┬──────────┬────────────┬────────────┬────────────┐
│ cms_certif ┆ payer  ┆ code  ┆ internal_r ┆ ... ┆ price    ┆ extracted_ ┆ extracted_ ┆ extracted_ │
│ ication_nu ┆ ---    ┆ ---   ┆ evenue_cod ┆     ┆ ---      ┆ ndc        ┆ cpt        ┆ drg        │
│ m          ┆ str    ┆ str   ┆ e          ┆     ┆ str      ┆ ---        ┆ ---        ┆ ---        │
│ ---        ┆        ┆       ┆ ---        ┆     ┆          ┆ list[str]  ┆ list[str]  ┆ list[str]  │
│ cat        ┆        ┆       ┆ str        ┆     ┆          ┆            ┆            ┆            │
╞════════════╪════════╪═══════╪════════════╪═════╪══════════╪════════════╪════════════╪════════════╡
│ 010055     ┆ MAX    ┆ 29240 ┆ NONE       ┆ ... ┆ 22445.67 ┆ null       ┆ ["29240"]  ┆ null       │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 033028     ┆ CAREWO ┆ NONE  ┆ 250        ┆ ... ┆ 8.23     ┆ null       ┆ n

In [1388]:
len(unmatched)/len(df)

0.5693638

In [1416]:
df.join(s, left_on = 'code', right_on = 'dx', how = 'outer').with_column(
    pl.when(
        col('drg').is_not_null() & col('extracted_drg').is_not_null()
    ).then(
        col('extracted_drg').arr.concat(col('drg'))
    ).otherwise(
        pl.when(
            col('drg').is_not_null()
        ).then(
            # col('extracted_drg')
            col('drg').apply(lambda x: [x])
        ).otherwise(None)
    )
)

ComputeError: Cannot cast list type

In [1004]:
cpt_marks   = ['CPT', 'HCPCS?/CPT', 'CPT/HCPCS?', 'HCPCS?']
drg_marks   = ['DRG', 'MS-DRG', 'MSD', 'MS']
ndc_marks   = ['NDC']
other_marks = ['CDM', 'PH\d+']

def list_to_regexp(items, b = True): 
    '''Create a regexp that matches any items in the list
    b = True: word boundaries included
    b = False: word boundaries excluded'''
    if b: return '|'.join([fr'\b{item}\b' for item in items])
    return '|'.join(items)

def in_column(colname, items, b = True): 
    '''Does the column contain any of the items?'''
    return col(colname).str.contains(list_to_regexp(items, b))

def extract_from(colname, items):
    '''Convenience function'''
    return col(f'{colname}').str.extract_all('|'.join(items))

def extract_cpt(df):
    '''
    Extract CPT singlet codes (i.e. not ranges) from a single column (usually 'code')
    '''
    
    # regexp to capture procedural terminology codes from 00100-99499
    # https://3widgets.com/: used this to generate the numeric aspect of the code
    
    # We make a column 'code_' where we'll put the partially cleaned data
    # The lack of positive/negative lookaheads in the rust regex parser
    # forces us to work in two passes.
    
    cpt_i_codes  = ['0010[0-9]','001[1-9][0-9]','00[2-9][0-9]{2}',
                    '0[1-9][0-9]{3}','[1-8][0-9]{4}','9[0-8][0-9]{3}',
                    '99[0-4][0-9]{2}']

    cpt_ii_codes = ['[ACEGJKLMQTV]\d{4}', '\d{4}[ACEGJKLMQTV]']
    
    code_ranges = ['([ACEGJKLMQTV]|\d)\d{3}([ACEGJKLMQTV]|\d)\s?-\s?([ACEGJKLMQTV]|\d)\d{3}([ACEGJKLMQTV]|\d)']
    
    cpt_found = in_column('code_', cpt_i_codes + cpt_ii_codes)
    other_marks_found = in_column('code_', drg_marks + ndc_marks + other_marks)
    
    df_ = df.with_columns([
        
        # Erase CPT code ranges (like 00100-00111, which confuse the regex) and create new column
        
        col('code').str.replace_all(list_to_regexp(cpt_ranges), '').alias('code_')
    
    ]).with_columns([
        
        # Extract CPT codes from column with ranges erased
        # This extracts any code like 00100 or A1921
        
        pl.when(cpt_found & ~other_marks_found).then(
            extract_from('code_', cpt_codes + cpt_ii_codes)
        ).otherwise(None).alias('cpts_extracted'),
        
        # Sometimes CPT/HCPCS is marked in the column. 
        # In this case, extract the code that follows the marker
        # This extracts codes like CPT1234 --> 1234
        
        pl.when(
            in_column('code_', cpt_marks, b = False) & 
            ~other_marks_found
        ).then(
            col('code').str.extract(r'\b(?:CPT|HCPCS?/CPT|CPT/HCPCS?|HCPCS?)\s?([A-Z]?\d{3,5}[A-Z]?)\b', 1)
        ).otherwise(None).apply(lambda x: [x]).alias('cpts_extracted_')
        
    ])
    
    # This is a workaround for what (I believe) is a polars bug
    if any(df_['cpts_extracted_'].is_not_null()): 
        df_ = df_.with_column(
            col('cpts_extracted').fill_null(col('cpts_extracted_'))
        )
        
    return df_.drop(['code_', 'cpts_extracted_'])
        
def extract_drg(df):
    '''
    Extract DRG singlet codes (i.e. not ranges) from multiple columns.
    Searches principally 'code' column, followed by 'internal_revenue_code'
    '''
    
    newcolname = 'glob'
    other_marks_found = in_column(newcolname, cpt_marks + ndc_marks + other_marks)
    
    df_ = df.with_column(
        
        pl
        .concat_str(['code', 'internal_revenue_code'], '~')
        .str.replace_all('\\bFY\\b|\\b\d{4,}\\b|\\bV\d{2}\\b|\(|\)', '')
        .alias('glob')
        
    ).with_columns([
        
        # Extract DRG codes from code column when the 
        # code column simply contains a 3 digit number
        
        pl.when(
            col('code').str.contains(r'^\d{1,3}(-\d{1,2})?$')
        ).then(
            col('code').str.extract(r'^(\d{1,3})(-\d{1,2})?$', 1)
        ).otherwise(None).alias('drg_extracted'),
        
        # Sometimes DRG is marked in the column. 
        # In this case, extract the code that follows the marker
        
        pl.when(
            in_column('glob', drg_marks, b = False) & ~other_marks_found
        ).then(
            col('code').str.extract(r'\b(?:MS|MSD)?(?:-|\s)?(?:DRG)?(\d{1,3})\b', 1)
        ).otherwise(None).alias('drg_extracted_')

    ]).with_column(
        col('drg_extracted').fill_null(col('drg_extracted_'))
    ).drop(['drg_extracted_', 'glob'])
    
    return df_

def extract_ndc(df):
    '''
    Extract NDC singlet codes (i.e. not ranges) from multiple columns.
    Searches principally 'code' column, followed by 'internal_revenue_code'
    and 'description'
    
    See: https://en.wikipedia.org/wiki/National_drug_code
    '''
    
    newcolname = 'glob'
    ndc_codes = ['0?\d{4}-\d{4}-\d{2}0?','0?\d{5}-\d{3}-\d{2}0?','0?\d{5}-\d{4}-\d{1}0?']
    
    df_ = df.with_column(
        
        pl
        .concat_str(['code', 'internal_revenue_code', 'description'], '~')
        .alias('glob')
        
    ).with_columns([
        
        # Extract NDC codes from code column when the 
        # code column contains an NDC string
        
        pl.when(
            in_column('code', ndc_codes)
        ).then(
            col('code').str.extract(list_to_regexp(ndc_codes))
        ).otherwise(None).alias('ndc_extracted'),
        
        # Sometimes NDC is marked in the column. 
        # In this case, extract the code that follows the marker
        # Can use more generous searching
        
        pl.when(
            in_column('glob', ['NDC'], b = False)
        ).then(
            col('glob').str.extract(r'([0-9-]{12})', 1)
        ).otherwise(None).alias('ndc_extracted_')

    ]).with_column(
        col('ndc_extracted').fill_null(col('ndc_extracted_'))
    ).drop(['ndc_extracted_', 'glob'])
    
    return df_
    
def extract_cpt_range(df):
    '''
    Extract CPT ranges from the 'code' column
    '''
    
    code_ranges = ['([ACEGJKLMQTV]|\d)\d{3}([ACEGJKLMQTV]|\d)\s?-\s?([ACEGJKLMQTV]|\d)\d{3}([ACEGJKLMQTV]|\d)']
    
    df_ = df.with_columns([
        
        pl.when(
            in_column('code', code_ranges)
        ).then(
            col('code').str.extract_all(list_to_regexp(code_ranges))
        ).otherwise(None).alias('cpt_ranges_extracted'),
        
    ])
    
    return df_